# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weatherpy_data = "..starter_code/weatherpy_data.csv"

weather_df = pd.read_csv(weatherpy_data.csv)

weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
#drop NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#ideal weather data frame
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions

# A max temperature lower than 80/ higher than 70
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

#wind less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

#no clouds
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

#drop null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info


In [ ]:
# Add marker layer ontop of heat map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

# Display figure
